In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list the files in the input directory

import os
print(os.listdir("../input"))

# Any results you write to the current directory are saved as output.

**Basic Perceptron Implementation**

First, we need to understand the basics of neural networks. A neural network consists of one or multiple layers of neurons, named after the biological neurons in human brains. We will demonstrate the mechanics of a single neuron by implementing a perceptron. In a perceptron, a single unit (neuron) performs all the computations.

In [ ]:
import numpy as np
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
# We will be using the Iris Plants Database
from sklearn.datasets import load_iris
SEED = 2017

# The first two classes (Iris-Setosa and Iris-Versicolour) are linear
iris = load_iris()
idxs = np.where(iris.target<2)
X = iris.data[idxs]
y = iris.target[idxs]

In [ ]:
plt.scatter(X[y==0][:,0],X[y==0][:,2], color='green', label='Iris-Setosa')
plt.scatter(X[y==1][:,0],X[y==1][:,2], color='red', label='Iris-Versica')
plt.title('Iris Plants Database')
plt.xlabel('sepal length in cm')
plt.ylabel('sepal width in cm')
plt.legend()
plt.show()

In [ ]:

def sigmoid(x):
    return 1 / (1 + np.exp(-x))

In [ ]:
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2)
weights = np.random.normal(size=X_train.shape[1])
bias = 1
learning_rate = 0.1
n_epochs = 15
del_w = np.zeros(weights.shape)
hist_loss = []
hist_accuracy = []

for i in range(n_epochs):
    # We apply a simple step function, if the output is > 0.5 we predict 1, else 0
    output = np.where((X_train.dot(weights)+bias)>0.5, 1, 0)

    # Compute MSE
    error = np.mean((y_train-output)**2)

    # Update weights and bias
    weights-= learning_rate * np.dot((output-y_train), X_train)
    bias += learning_rate * np.sum(np.dot((output-y_train), X_train))

    # Calculate MSE
    loss = np.mean((output - y_train) ** 2)
    hist_loss.append(loss)

    # Determine validation accuracy
    output_val = np.where(X_val.dot(weights)>0.5, 1, 0)
    accuracy = np.mean(np.where(y_val==output_val, 1, 0))
    hist_accuracy.append(accuracy)

In [ ]:
fig = plt.figure(figsize=(8, 4))
a = fig.add_subplot(1,2,1)
imgplot = plt.plot(hist_loss)
plt.xlabel('epochs')
a.set_title('Training loss')

a=fig.add_subplot(1,2,2)
imgplot = plt.plot(hist_accuracy)
plt.xlabel('epochs')
a.set_title('Validation Accuracy')
plt.show()

**Multiplayer Perceptrons Using Keras and Handwritten Digit Recognizer Dataset**

A neural network consists of one or multiple layers of neurons, named after the biological neurons in human brains. We will demonstrate the mechanics of a single neuron by implementing a perceptron. In a perceptron, a single unit (neuron) performs all the computations.

In [ ]:
import numpy as np
import pandas as pd
from keras.datasets import mnist
from keras.models import Sequential
from keras.layers.core import Dense, Activation
from keras.optimizers import SGD
from keras.utils import np_utils as kutils
from sklearn.model_selection import train_test_split

np.random.seed(1671) # for reproducibility
# network and training parameters
NB_EPOCH = 200
BATCH_SIZE = 128
VERBOSE = 1
NB_CLASSES = 10 # number of outputs = number of digits
RESHAPED = 784
OPTIMIZER = SGD() # SGD optimizer, explained later in this chapter
N_HIDDEN = 128
VALIDATION_SPLIT=0.2 # how much TRAIN is reserved for VALIDATION
# data: shuffled and split between train and test sets
#

def load_dataset(train_path='../input/digit-recognizer/train.csv',test_path='../input/digit-recognizer/test.csv'):
    #global train,test,trainX,trainY,nb_classes
    img_rows=28
    img_cols=28
    train = pd.read_csv(train_path).values # produces numpy array
    test  = pd.read_csv(test_path).values # produces numpy array
    print("Train Shape :",train.shape)
    print("Test Shape :",test.shape)
    trainX = train[:, 1:].reshape(train.shape[0], RESHAPED)
    trainX = trainX.astype(float)
    trainX /= 255.0
    trainY = kutils.to_categorical(train[:, 0])
    nb_classes = trainY.shape[1]
    print("TrainX Shape : ",trainX.shape)
    print("Trainy shape : ",trainY.shape)
    testX = test.reshape(test.shape[0], RESHAPED)
    testX = testX.astype(float)
    testX /= 255.0
    trainY = kutils.to_categorical(train[:, 0])
    return trainX,trainY,testX,nb_classes

X,y,test,nb_classes=load_dataset()
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=42)

In [ ]:
#X_train is 60000 rows of 28x28 values --> reshaped in 60000 x 784

# final stage is softmax
model = Sequential()
model.add(Dense(NB_CLASSES, input_shape=(RESHAPED,)))
model.add(Activation('softmax'))
model.summary()

In [ ]:
model.compile(loss='categorical_crossentropy', optimizer=OPTIMIZER, metrics=['accuracy'])

In [ ]:

history = model.fit(X_train, y_train, batch_size=BATCH_SIZE, epochs=NB_EPOCH, verbose=VERBOSE, 
                    validation_split=VALIDATION_SPLIT)
score = model.evaluate(X_test, y_test, verbose=VERBOSE)
print("Test score:", score[0])
print('Test accuracy:', score[1])

**Improving the Simple Neural Net in Keras with Hidden Layers**

A first improvement is to add additional layers to our network. So, after the input layer, we have a first dense layer with the N_HIDDEN neurons and an activation function relu. This additional layer is considered hidden because it is not directly connected to either the input or the output. After the first hidden layer, we have a second hidden layer, again with the N_HIDDEN neurons, followed by an output layer with 10 neurons, each of which will fire when the relative digit is recognized. 

In [ ]:
# final stage is softmax
model = Sequential()
model.add(Dense(N_HIDDEN, input_shape=(RESHAPED,)))
model.add(Activation('relu'))
model.add(Dense(N_HIDDEN))
model.add(Activation('relu'))
model.add(Dense(NB_CLASSES))
model.add(Activation('softmax'))
model.summary()
model.compile(loss='categorical_crossentropy', optimizer=OPTIMIZER, metrics=['accuracy'])

In [ ]:
history = model.fit(X_train, y_train,batch_size=BATCH_SIZE, epochs=NB_EPOCH,
                    verbose=VERBOSE, validation_split=VALIDATION_SPLIT)

score = model.evaluate(X_test, y_test, verbose=VERBOSE)
print("Test score:", score[0])
print('Test accuracy:', score[1])

**Further Improving the Simple Net in Keras with Dropout**

A second improvement is very simple. We decide to randomly drop with the dropout probability some of the values
propagated inside our internal dense network of hidden layers. In machine learning, this is a well-known form of regularization. Surprisingly enough, this idea of randomly dropping a few values can significantly improve classification performance:

In [ ]:
from keras.layers.core import Dropout
DROPOUT = 0.3
# M_HIDDEN hidden layers 10 outputs
model = Sequential()
model.add(Dense(N_HIDDEN, input_shape=(RESHAPED,)))
model.add(Activation('relu'))
model.add(Dropout(DROPOUT))
model.add(Dense(N_HIDDEN))
model.add(Activation('relu'))
model.add(Dropout(DROPOUT))
model.add(Dense(NB_CLASSES))
model.add(Activation('softmax'))
model.summary()
model.compile(loss='categorical_crossentropy',optimizer=OPTIMIZER,metrics=['accuracy'])
history = model.fit(X_train, y_train, batch_size=BATCH_SIZE, epochs=NB_EPOCH,verbose=VERBOSE,
                    validation_split=VALIDATION_SPLIT)

score = model.evaluate(X_test, y_test, verbose=VERBOSE)
print("Test score:", score[0])
print('Test accuracy:', score[1])

**Perceptron Algorithm from Sklearn on Digits Dataset**


In [ ]:
from sklearn.datasets import load_digits
from sklearn.linear_model import Perceptron
from sklearn.metrics import f1_score, classification_report

X, y = load_digits(return_X_y=True)
clf = Perceptron(tol=1e-3, random_state=0)
clf.fit(X, y)
print("Classification Score : ",clf.score(X, y))
predictions = clf.predict(X)
print(classification_report(y, predictions))

**Perceptron Using Keras  on Digits Dataset**

In [ ]:
from sklearn.datasets import load_digits
from sklearn.linear_model import Perceptron
from sklearn.metrics import f1_score, classification_report
from keras.models import Sequential
from keras.layers.core import Dense, Activation

X, y = load_digits(return_X_y=True)
#digits = load_digits()

print(len(X))
print(len(y))

# final stage is softmax
model = Sequential()
# Input - Layer
model.add(Dense(50, activation = "relu", input_shape=(64,)))
# Output- Layer
model.add(Dense(10, activation = "sigmoid"))

model.summary()
model.compile(optimizer = "adam",loss = "binary_crossentropy", metrics = ["accuracy"])

results = model.fit(X, y, epochs= 20, batch_size = 500)

**Iris Recognition Using Perceptron with Scikit Learn**

In [ ]:
# Load required libraries
from sklearn import datasets
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import Perceptron
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.metrics import f1_score, classification_report
import numpy as np

# Load the iris dataset
iris = datasets.load_iris()

# Create our X and y data
X = iris.data
y = iris.target

# Train the scaler, which standarizes all the features to have mean=0 and unit variance
sc = StandardScaler()
sc.fit(X)

# Apply the scaler to the X training data
X_std = sc.transform(X)
# Split the data into 70% training data and 30% test data
X_train, X_test, y_train, y_test = train_test_split(X_std, y, test_size=0.3)

# Create a perceptron object with the parameters: 40 iterations (epochs) over the data, and a learning rate of 0.1
model = Perceptron(n_iter=40, eta0=0.1, random_state=0,activation='relu')

# Train the perceptron
model.fit(X_train, y_train)

# Apply the trained perceptron on the X data to make predicts for the y test data
y_pred = model.predict(X_test)

print(classification_report(y_test, y_pred))

**Multiplayer Perceptrons Using Sklearn and Handwritten Digit Recognizer Dataset**

In [ ]:
import numpy as np
import pandas as pd
from keras.utils import np_utils as kutils
from sklearn.model_selection import train_test_split
from sklearn.neural_network import MLPClassifier

np.random.seed(1671) # for reproducibility
# network and training parameters
NB_EPOCH = 100
RESHAPED = 784
#BATCH_SIZE = 128
#VERBOSE = 1
#NB_CLASSES = 10 # number of outputs = number of digits
#OPTIMIZER = SGD() # SGD optimizer, explained later in this chapter
#N_HIDDEN = 128
#VALIDATION_SPLIT=0.2 # how much TRAIN is reserved for VALIDATION
# data: shuffled and split between train and test sets
#

def load_dataset(train_path='../input/digit-recognizer/train.csv',test_path='../input/digit-recognizer/test.csv'):
    #global train,test,trainX,trainY,nb_classes
    img_rows=28
    img_cols=28
    train = pd.read_csv(train_path).values # produces numpy array
    test  = pd.read_csv(test_path).values # produces numpy array
    print("Train Shape :",train.shape)
    print("Test Shape :",test.shape)
    trainX = train[:, 1:].reshape(train.shape[0], RESHAPED)
    trainX = trainX.astype(float)
    trainX /= 255.0
    trainY = kutils.to_categorical(train[:, 0])
    nb_classes = trainY.shape[1]
    print("TrainX Shape : ",trainX.shape)
    print("Trainy shape : ",trainY.shape)
    testX = test.reshape(test.shape[0], RESHAPED)
    testX = testX.astype(float)
    testX /= 255.0
    trainY = kutils.to_categorical(train[:, 0])
    return trainX,trainY,testX,nb_classes

X,y,test,nb_classes=load_dataset()
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=42)

In [ ]:
clf = MLPClassifier(hidden_layer_sizes=(100,100,100), max_iter=NB_EPOCH, alpha=0.0001,
                     solver='sgd', verbose=True,  random_state=21,tol=0.000000001)
clf.fit(X_train, y_train)
print("Classification Score : ",clf.score(X_train, y_train))

predictions = clf.predict(X_test)
print(classification_report(y_test, predictions))